# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
# import libraries
from sqlalchemy import create_engine 
import numpy as np
import pandas as pd

# To use regex
# Importing re
import re

# To Process Strings 
#import nltk
import nltk
nltk.download('punkt')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier


[nltk_data] Downloading package punkt to /Users/Hussain/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/Hussain/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///DB_messeges.db')
df = pd.read_sql_table("T_messeges", engine)
X = df.iloc[:,0]
Y = df.iloc[:,3:]

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    
    """This function normalize the string, remove punctuations, and then create a list of tokens. 
    Args:
        text: a string 
    Returns:
        processed_list : a processed list of strings (normalized, punctuations free, and tokenized)
    """
    
    # Convert string to lowercase
    text = text.lower()
    
    # Remove punctuations
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    # Tokenize words
    tokenized_words = word_tokenize(text)
    
    stemer = SnowballStemmer("english")
    processed_list = [stemer.stem(token) for token in tokenized_words if token not in stopwords.words("english")]
    
    return processed_list    

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [5]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 12)
pipeline.fit(X_train, Y_train)

/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 t

/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 t

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
pred_y =pipeline.predict(X= X_test)

In [8]:
for i in np.arange(0,35):
    print(Y_test.columns[i],classification_report(y_pred= pred_y[:,i], y_true= Y_test.iloc[:,i]))

related               precision    recall  f1-score   support

           0       0.64      0.48      0.55      1516
           1       0.85      0.92      0.88      4991

    accuracy                           0.82      6507
   macro avg       0.75      0.70      0.72      6507
weighted avg       0.80      0.82      0.81      6507

request               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5410
           1       0.79      0.46      0.58      1097

    accuracy                           0.89      6507
   macro avg       0.84      0.72      0.76      6507
weighted avg       0.88      0.89      0.88      6507

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6479
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


transport               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6211
           1       0.73      0.06      0.12       296

    accuracy                           0.96      6507
   macro avg       0.84      0.53      0.55      6507
weighted avg       0.95      0.96      0.94      6507

buildings               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6150
           1       0.73      0.11      0.19       357

    accuracy                           0.95      6507
   macro avg       0.84      0.55      0.58      6507
weighted avg       0.94      0.95      0.93      6507

electricity               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6365
           1       0.55      0.04      0.08       142

    accuracy                           0.98      6507
   macro avg       0.76      0.52      0.53      6507
weighted avg       0.97      0.98      0.9

### 6. Improve your model
Use grid search to find better parameters. 

In [11]:
parameters = {'vect__min_df': [1, 3],
              'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators':[4,8], 
              'clf__estimator__min_samples_split':[10,25]}

cv = GridSearchCV(pipeline, param_grid=parameters)

In [12]:
cv.fit(X_train, Y_train)

/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [14]:
cv_pred_y =cv.best_estimator_.predict(X= X_test)

In [15]:
for i in np.arange(0,35):
    print(Y_test.columns[i],classification_report(y_pred= cv_pred_y[:,i], y_true= Y_test.iloc[:,i]))

related               precision    recall  f1-score   support

           0       0.67      0.44      0.53      1516
           1       0.85      0.93      0.89      4991

    accuracy                           0.82      6507
   macro avg       0.76      0.69      0.71      6507
weighted avg       0.80      0.82      0.80      6507

request               precision    recall  f1-score   support

           0       0.90      0.97      0.93      5410
           1       0.76      0.49      0.59      1097

    accuracy                           0.89      6507
   macro avg       0.83      0.73      0.76      6507
weighted avg       0.88      0.89      0.88      6507

offer               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6479
           1       0.00      0.00      0.00        28

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99      650

/Users/Hussain/anaconda3/envs/dataScience/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


transport               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6211
           1       0.69      0.13      0.22       296

    accuracy                           0.96      6507
   macro avg       0.83      0.56      0.60      6507
weighted avg       0.95      0.96      0.94      6507

buildings               precision    recall  f1-score   support

           0       0.96      1.00      0.98      6150
           1       0.75      0.21      0.33       357

    accuracy                           0.95      6507
   macro avg       0.85      0.60      0.65      6507
weighted avg       0.94      0.95      0.94      6507

electricity               precision    recall  f1-score   support

           0       0.98      1.00      0.99      6365
           1       0.57      0.09      0.16       142

    accuracy                           0.98      6507
   macro avg       0.77      0.54      0.57      6507
weighted avg       0.97      0.98      0.9

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.